# Day One

## Initial Setup

In [1]:
%%time

# Imports.
import os
import requests

from dotenv import load_dotenv
from bs4 import BeautifulSoup

from IPython.display import Markdown, display
from openai import OpenAI

CPU times: user 781 ms, sys: 92.4 ms, total: 874 ms
Wall time: 953 ms


## Connecting to OpenAI

In [3]:
# Load openai key.
api_key = os.environ.get("OPENAI_API_KEY")

# Check the API key.
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")

elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")

elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")

else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
%%time

openai = OpenAI()

CPU times: user 85.1 ms, sys: 15.8 ms, total: 101 ms
Wall time: 101 ms


## Quick Call to a Frontier Model

In [5]:
%%time

# To give you a preview: 
# Calling OpenAI with these messages is this easy. 
# Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"

response = openai.chat.completions.create(
    model="gpt-4o-mini", 
    messages=[{
        "role": "user", 
        "content": message
    }]
)

print(response.choices[0].message.content)

Hello! Welcome! I'm glad to hear from you. How can I assist you today?
CPU times: user 22.7 ms, sys: 7.89 ms, total: 30.6 ms
Wall time: 1.02 s


## Simple Web Scrapper

In [6]:
# A class to represent a Webpage.
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": 
 "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library.
        """
        
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
%%time

# Let's try one out. Change the website and add print statements to follow along.
ed = Website("https://edwarddonner.com")

print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of Prompts

In [8]:
# Define our system prompt - you can experiment with this later, changing the last sentence 
# to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown in Brazilian Portuguese."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
   
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
   
    return user_prompt

In [10]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows;     please provide a short summary of this website in markdown.     If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired i

## Messages

In [11]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [12]:
%%time

# To give you a preview -- calling OpenAI with system and user messages:
response = openai.chat.completions.create(
    model="gpt-4o-mini", 
    messages=messages
)

print(response.choices[0].message.content)

Oh, let me think... it’s a tough one. Drumroll please... it’s 4! Shocking, I know.
CPU times: user 11.7 ms, sys: 7.26 ms, total: 19 ms
Wall time: 1.42 s


## Function for Useful Messages

In [13]:
# See how this function creates exactly the format above.
def messages_for(website):
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
# Try this out, and then try for a few more websites.
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown in Brazilian Portuguese.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows;     please provide a short summary of this website in markdown.     If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field w

## Bring it Together

In [15]:
# And now: call the OpenAI API. You will get very familiar with this!
def summarize(url):
    
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )

    return response.choices[0].message.content

In [16]:
%%time

print(summarize("https://edwarddonner.com"))

# Resumo do Site - Edward Donner

O site de Edward Donner apresenta seu trabalho e interesses em áreas como programação, inteligência artificial (IA) e produção de música eletrônica. Ed é o co-fundador e CTO da Nebula.io, uma empresa focada em aplicar IA na descoberta de talentos e no gerenciamento de recursos humanos. Anteriormente, ele fundou a startup untapt, adquirida em 2021.

## Análises Recentes e Recursos
- **23 de janeiro de 2025**: Workshop "LLM – Mãos à obra com Agentes" – Recursos disponíveis.
- **21 de dezembro de 2024**: Boas-vindas aos SuperDataScientists!
- **13 de novembro de 2024**: Recursos para "Dominando Engenharia de IA e LLM".
- **16 de outubro de 2024**: Recursos para a transição de "Engenheiro de Software para Cientista de Dados de IA".

O site enfatiza a interação e a conexão com outros profissionais interessados em LLMs (Modelos de Linguagem de Grande Escala) e oferece recursos úteis sobre o assunto.
CPU times: user 51.7 ms, sys: 341 μs, total: 52 ms
Wall tim

In [17]:
# A function to display this nicely in the Jupyter output, using markdown.
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
%%time

display_summary("https://edwarddonner.com")

# Resumo do site - Edward Donner

Este site é mantido por Ed, um entusiasta de programação e LLMs (Modelos de Linguagem de Grande Escala). Ele é co-fundador e CTO da Nebula.io, uma empresa que utiliza IA para ajudar pessoas a descobrirem seu potencial e se conectarem com oportunidades de trabalho. Com uma experiência anterior como fundador e CEO da untapt, Ed possui um forte histórico em tecnologia e inovação na área de talentos e recrutamento.

## Anúncios e novidades
- **23 de janeiro de 2025**: Workshop sobre LLM – Recursos práticos com agentes.
- **21 de dezembro de 2024**: Boas-vindas aos SuperDataScientists!
- **13 de novembro de 2024**: Recursos para dominar engenharia e inteligência artificial de LLM.
- **16 de outubro de 2024**: Recursos sobre a transição de Engenheiro de Software para Cientista de Dados em IA. 

Ed também compartilha recursos, reflexões e experiências sobre suas áreas de interesse, convidando os visitantes a se conectarem com ele.

CPU times: user 51 ms, sys: 0 ns, total: 51 ms
Wall time: 5.13 s


## Other Websites

In [19]:
%%time

display_summary("https://cnn.com")

# Resumo do site da CNN

O site da CNN oferece uma ampla variedade de notícias e vídeos sobre diversos tópicos, incluindo política, negócios, saúde, entretenimento, ciência e eventos globais. A plataforma está focada em reportagens atualizadas, análises e conteúdos multimídia.

## Principais notícias recentes:
- **Guerra na Ucrânia e Israel-Hamas**: A CNN cobre eventos relacionados à guerra na Ucrânia e aos conflitos no Oriente Médio, incluindo a troca de acusações entre Hamas e Israel sobre violações de cessar-fogo.
- **Política dos EUA**: O ex-presidente Trump está no centro de várias reportagens, incluindo sua administração que lançou uma blitz de aplicação de imigração em Chicago e suas tensões com o presidente da Colômbia, que resultaram em tarifas de retaliação.
- **Eventos Educacionais e Culturais**: Destaques incluem uma exposição de moda no Louvre e uma nova análise sobre fosséis encontrados na China que podem alterar a compreensão da história humana.
- **Saúde e Bem-Estar**: Relatórios sobre novas diretrizes alimentares que avaliam os riscos e benefícios do consumo de álcool estão disponíveis, assim como informações sobre a vida com ADHD e suas implicações para a saúde mental.

Além disso, o site fornece acesso a vídeos ao vivo, programas e podcasts, permitindo que os usuários se mantenham atualizados com as notícias mais relevantes do momento.

CPU times: user 384 ms, sys: 80 ms, total: 464 ms
Wall time: 6.6 s


In [20]:
%%time

display_summary("https://anthropic.com")

# Resumo do Site da Anthropic

O site da Anthropic apresenta a empresa como uma entidade focada em pesquisa e segurança em inteligência artificial, com sede em San Francisco. Destaca-se pelo desenvolvimento de modelos de IA, especialmente o Claude 3.5 Sonnet, que é seu modelo mais inteligente até o momento.

## Principais Conteúdos

- **Claude**: O principal produto da empresa, com ênfase na construção de aplicações e experiências personalizadas utilizando o Claude.
- **Anúncios Recentes**:
  - **22 de Outubro de 2024**: Introdução ao uso de computadores, incluindo lançamentos do modelo Claude 3.5 Sonnet e Claude 3.5 Haiku.
  - **4 de Setembro de 2024**: Lançamento do Claude para o setor empresarial.
  - **15 de Dezembro de 2022**: Pesquisa sobre "Constitutional AI: Harmlessness from AI Feedback".
  - **8 de Março de 2023**: Discussão sobre as visões centrais da empresa em segurança em IA.

## Visão Geral

Com uma equipe interdisciplinar com experiência em aprendizado de máquina, física, políticas e produtos, a Anthropic busca criar sistemas de IA confiáveis e benéficos.

CPU times: user 50.5 ms, sys: 7.04 ms, total: 57.6 ms
Wall time: 4.96 s


## Exercise

In [21]:
LFA_URL = "https://lfa.if.usp.br/p%C3%A1gina-inicial"
lfa = Website(LFA_URL)

In [22]:
%%time

# Step 1: Create your prompts.
system_prompt = \
    f"""
    Você é um assistente que analisa o conteúdo de um site e fornece um resumo curto,
    ignorando o texto que pode ser relacionado à navegação. Responda em markdown em português brasileiro.
    """
user_prompt = f"""
    Voce está olhando para um site intitulado {lfa.title}.
    O conteúdo deste site é o seguinte; por favor, forneça um breve resumo deste site em markdown em Português Brasileiro.

    Conteúdo do site:
    {lfa.text}
"""

# Step 2: Make the messages list.
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call OpenAI.
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)
results = response.choices[0].message.content

# Step 4: print the result.
print(results)

# Resumo do Site LFA - Laboratório de Física Atmosférica

O Laboratório de Física Atmosférica (LFA) se dedica ao estudo das propriedades físicas e químicas da atmosfera, focando na análise do ciclo de vida de partículas atmosféricas, como aerossóis e nuvens. As pesquisas abrangem áreas urbanas, como São Paulo, e a Amazônia, lidando com questões de poluição e mudanças climáticas.

## Áreas de Pesquisa
- **Aerossóis Naturais na Amazônia**: Estudo das emissões de compostos orgânicos voláteis e sua transformação em aerossóis, fundamentais para a formação de nuvens.
- **Aerossóis de Queimadas**: Análise das emissões de queimadas na Amazônia e seus impactos no clima e na saúde.
- **Mudanças Climáticas**: Investigação do equilíbrio radiativo da Terra e as consequências de alterações de uso do solo e emissões de gases.
- **Vapor de Água e Nuvens**: Estudo do transporte de vapor de água e suas implicações para a precipitação em regiões continentais.
- **Interação entre Nuvens e Aerossóis**: Pes

In [23]:
%%time

display_summary(LFA_URL)

# Resumo do Laboratório de Física Atmosférica (LFA)

O **Laboratório de Física Atmosférica (LFA)** se dedica ao estudo das propriedades físicas e químicas da atmosfera, com foco em temas como o ciclo de vida de aerossóis e nuvens, especialmente em contextos urbanos e na Amazônia. O laboratório realiza pesquisas sobre poluição urbana em cidades brasileiras e as implicações da queima de biomassa na Amazônia, analisando suas consequências para o clima e a saúde pública.

## Novidades

- **CPAM 2024**: Anúncio da Conferência Pan-Americana de Meteorologia, a ser realizada em 2024.
- **Entrevista com Paulo Artaxo**: Discussão sobre a crise climática e ações governamentais em uma reportagem do Jornal da USP.
- **Apresentação sobre o CEAS**: Foco na integração da ciência com a Amazônia Sustentável.

## Áreas de Pesquisa

1. **Aerossóis Naturais na Amazônia**: Estudo da emissão de compostos orgânicos voláteis pelas florestas e seu papel na formação de nuvens.
2. **Aerossóis de Queimadas**: Análise das emissões de queimadas e seus efeitos climáticos e na saúde pública.
3. **Mudanças Climáticas Globais**: Pesquisa sobre a influência do desmatamento e das emissões de gases de efeito estufa no clima.
4. **Vapor de Água e Nuvens**: Investigação do transporte de vapor de água e seus impactos na precipitação.
5. **Interação entre Nuvens e Aerossóis**: Estudo de como aerossóis influenciam a formação de nuvens.
6. **Poluição Urbana**: Análise da poluição do ar nas cidades e sua relação com a saúde.

O LFA, ligado ao Instituto de Física da Universidade de São Paulo, desenvolve uma série de projetos experimentais e utiliza ferramentas avançadas de análise, como sensoriamento remoto e modelos numéricos.

CPU times: user 78.6 ms, sys: 11.9 ms, total: 90.5 ms
Wall time: 12.7 s
